In [4]:
import os
import numpy as np
import pandas as pd

---
1. 预处理
---
'柚宝宝孕育'App数据有问题，应该删掉

---

In [10]:
usage_file = '../resources/data0117'

In [17]:
usage_rdd = sc.textFile(usage_file).map(lambda x: x.split("|"))
usage_rdd = usage_rdd.filter(lambda x: x[2] != '柚宝宝孕育')

In [18]:
usage_rdd.take(5)

[['C966BB7B4BB196602D1054B27E6F9BAE', '10', 'QQ', '通讯聊天', '15'],
 ['C2827EBF2B61C4FB71C164F3D13BE312', '08', '微信', '通讯聊天', '1'],
 ['F819FE792867382C9C8BD39BA7837746', '21', 'QQ', '通讯聊天', '4'],
 ['DA549AD85F2DD7E87126416CCA4C44B4', '18', '微信', '通讯聊天', '3'],
 ['F81A9E1BCC2FF6820539952284E9C7A9', '12', '开心消消乐', '游戏服务', '1']]

In [19]:
usage_rdd.count()

949908

### 构建索引，对特征进行有序编码

In [21]:
userlist = usage_rdd.map(lambda x: x[0]).distinct().collect()
applist = usage_rdd.map(lambda x: x[2]).distinct().collect()
catlist = usage_rdd.map(lambda x: x[3]).distinct().collect()
userlist.sort()
applist.sort()
catlist.sort()
len(userlist), len(applist), len(catlist)

(25413, 54, 19)

In [22]:
nodelist = userlist + applist + catlist
node2id = dict(zip(nodelist, range(len(nodelist))))
# 序列化保存编码
import pickle
pickle.dump(node2id, open('../resources/node2id.pkl', 'wb'))

In [36]:
rdd = usage_rdd.map(lambda x: (node2id[x[0]], x[2], x[3])).distinct()  # userid, app, cat
user_app = rdd.map(lambda x: '%s\tU\t%s\tA\tU-A' % (x[0], x[1])).distinct()
app_user = rdd.map(lambda x: '%s\tA\t%s\tU\tA-U' % (x[1], x[0])).distinct()
app_cat = rdd.map(lambda x: '%s\tA\t%s\tC\tA-C' % (x[1], x[2])).distinct()
cat_app = rdd.map(lambda x: '%s\tC\t%s\tA\tC-A' % (x[2], x[1])).distinct()

In [37]:
rdd.first()

(13368, '微信', '通讯聊天')

In [45]:
print(user_app.first())
print(app_user.first())
print(app_cat.first())
print(cat_app.first())
print(user_app.count(), app_user.count(), app_cat.count(), cat_app.count())

24544	U	支付宝	A	U-A
开心消消乐	A	6288	U	A-U
微信	A	通讯聊天	C	A-C
通讯聊天	C	微信	A	C-A
192963 192963 54 54


In [46]:
out = user_app.union(app_user)
out = out.union(app_cat)
out = out.union(cat_app)
print(out.count())
out.repartition(1).saveAsTextFile('./graph_resources/edges4hin2vec')

386034


In [47]:
(192963 +54)*2

386034

In [48]:
# rename of edges4hin2vec/part-00000
os.system('mv graph_resources/edges4hin2vec/part-00000 graph_resources/edges4hin2vec.txt')

0